In [1]:
import numpy as np
import networkx as nx

import mmk

In [2]:
simple_com_prt = mmk.MarkovChain(
    {
        "start": {
            "try": 1.
        },
        "try": {
            "lost": .1,
            "delivered": .9,
        },
        "lost": {
            "try": 1.,
        },
        "delivered": {
            "start": 1.,
        },
    }
)

In [3]:
formula = "eventually delivered"  # or "<> delivered" 
p = mmk.probability_of_formula(simple_com_prt, formula)

for s, p in simple_com_prt.vector_to_state_weights(p).items():
    print(f"ℙ ( {s} ⊨ {formula} ) = {p}")

ℙ ( delivered ⊨ eventually delivered ) = 1.0
ℙ ( lost ⊨ eventually delivered ) = 1.0000000000000036
ℙ ( start ⊨ eventually delivered ) = 0.999999999999996
ℙ ( try ⊨ eventually delivered ) = 1.0


In [4]:
p = .05
q = .1
zeroconf_prt = mmk.MarkovChain(
    {
        "start": {
            "s1": q,
            "ok": 1 - q,
        },
        "s1": {
            "start": 1 - p,
            "s2": p,
        },
        "s2": {
            "start": 1 - p,
            "s3": p,
        },
        "s3": {
            "start": 1 - p,
            "s4": p,
        },
        "s4": {
            "start": 1 - p,
            "error": p,
        },
        "error": {
            "error": 1.,
        },
        "ok": {
            "ok": 1.,
        },
    }
)

In [5]:
formula = "eventually ok"  # or "<> ok" 
p = mmk.probability_of_formula(zeroconf_prt, formula)

for s, p in zeroconf_prt.vector_to_state_weights(p).items():
    print(f"ℙ ( {s} ⊨ {formula} ) = {p}")

ℙ ( error ⊨ eventually ok ) = 0.0
ℙ ( ok ⊨ eventually ok ) = 1.0
ℙ ( s1 ⊨ eventually ok ) = 0.9999930555603785
ℙ ( s2 ⊨ eventually ok ) = 0.9998743056428434
ℙ ( s3 ⊨ eventually ok ) = 0.9974993072921479
ℙ ( s4 ⊨ eventually ok ) = 0.9499993402782358
ℙ ( start ⊨ eventually ok ) = 0.9999993055560384
